### Notebook for generating heterogeneous server configurations

Generate XML server configurations with different server capacity and specifications.

In [19]:
import pickle 
import json 
import shutil 
import os 
import math 
import random 
import numpy as np 
from lxml import etree 
from collections import Counter


RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [20]:

with open('../AP_locations_1100m_45m.pickle', 'rb') as f: 
    AP_coords = pickle.load(f)

with open('../Server_locations_20_1100m_45m.pickle', 'rb') as f: 
    s_coords_20 = pickle.load(f)

with open('../Server_locations_100_1100m_45m.pickle', 'rb') as f: 
    s_coords_100 = pickle.load(f)

with open('../AP_links_1100m_45m.pickle', 'rb') as f: 
    links = pickle.load(f)


print("Loaded generated topology and placement data")
print(f"Loaded {len(AP_coords)} AP locations\n")
print(f"Loaded {len(s_coords_20)} high capacity edge server locations\n")
print(f"Loaded {len(s_coords_100)} low capacity edge server locations\n")
print(f"Loaded {len(links)} topology links\n")

Loaded generated topology and placement data
Loaded 247 AP locations

Loaded 20 high capacity edge server locations

Loaded 100 low capacity edge server locations

Loaded 246 topology links



In [21]:
with open('../server20_cluster_info_centralized.json', 'r', encoding='utf-8') as f:
    server20_cluster_info_centralized = json.load(f)

with open('../server20_cluster_info_hybrid.json', 'r', encoding='utf-8') as f:
    server20_cluster_info_hybrid = json.load(f)

with open('../server100_cluster_info_centralized.json', 'r', encoding='utf-8') as f:
    server100_cluster_info_centralized = json.load(f)

with open('../server100_cluster_info_hybrid.json', 'r', encoding='utf-8') as f:
    server100_cluster_info_hybrid = json.load(f)

print("Loaded cluster information for different control topologies")

Loaded cluster information for different control topologies


### Define Heterogeneous Server Specifications

Three server classes with different capacities and power consumption profiles:


In [22]:

import json 
with open('heter_server_configs.json', 'r') as f: 
    config = json.load(f)


server_specs_20 = config['server_specs_20']
server_specs_100 = config['server_specs_100']

total_mips_20 = sum(spec['mips'] * spec['count'] for spec in server_specs_20.values())
total_mips_100 = sum(spec['mips'] * spec['count'] for spec in server_specs_100.values())

print(f"Hetereogeneous 20-server scenario total MIPS: {total_mips_20}")
print(f"Distribution: Class A: {server_specs_20['A']['count']}, Class B: {server_specs_20['B']['count']}, Class C: {server_specs_20['C']['count']}")
print(f"Hetereogeneous 100-server scenario total MIPS: {total_mips_100}")
print(f"Distribution: Class A: {server_specs_20['A']['count']}, Class B: {server_specs_20['B']['count']}, Class C: {server_specs_20['C']['count']}")

Hetereogeneous 20-server scenario total MIPS: 400000
Distribution: Class A: 6, Class B: 8, Class C: 6
Hetereogeneous 100-server scenario total MIPS: 980000
Distribution: Class A: 6, Class B: 8, Class C: 6


### Server class assignment 

We randomly assign a server class to a server position

In [23]:
def assign_server_classes(num_servers, spec_dict): 
    classes = []
    for class_name, specs in spec_dict.items():
        classes.extend([class_name] * specs['count'])

    assert len(classes) == num_servers, f"Class count mismatch {len(classes)} != {len(num_servers)}"

    random.shuffle(classes)
    return classes



server_classes_20 = assign_server_classes(20, spec_dict=server_specs_20)
server_classes_100 = assign_server_classes(100, spec_dict=server_specs_100)
print("\n20-Server class distribution")
print(Counter(server_classes_20))
print(f"Assignment: {server_classes_20}")
print(f"\n100-Server class distribution")
print(Counter(server_classes_100))
print(f"Assignment: {server_classes_100}")



20-Server class distribution
Counter({'B': 8, 'C': 6, 'A': 6})
Assignment: ['C', 'A', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'B', 'C', 'B', 'A', 'B', 'A', 'C', 'B', 'B', 'A', 'A']

100-Server class distribution
Counter({'B': 50, 'C': 30, 'A': 20})
Assignment: ['A', 'C', 'C', 'C', 'B', 'A', 'C', 'C', 'A', 'C', 'C', 'B', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B', 'C', 'B', 'C', 'A', 'A', 'C', 'B', 'B', 'C', 'B', 'C', 'C', 'B', 'B', 'C', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'C', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'A', 'B', 'B', 'C', 'B', 'A', 'B', 'A', 'B', 'C', 'A', 'B', 'B', 'B', 'A', 'C', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'A', 'C', 'B', 'A', 'C', 'B', 'B', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'B', 'C']


### Server Specification Generator

In [24]:
def server_specification_generator(server_classes, specs_dict):
    """
    Creates a generator that yields server specifications based on assigned classes.
    
    This generator cycles through servers in order, yielding specs for each based on its class.
    
    Args:
        server_classes: List of class assignments for each server position
        specs_dict: Dictionary of specifications for each class
        
    Returns:
        Generator function that yields specifications in the required order
    """
    server_index = 0
    
    def generator():
        nonlocal server_index
        
        # Get the class for this server
        server_class = server_classes[server_index % len(server_classes)]
        specs = specs_dict[server_class]
        
        # Increment index immediately so next call gets next server
        # (This is safe because server_class is already retrieved for this call)
        server_index += 1
        
        # Yield specifications in the order expected by the XML generation
        yield 'True'  # periphery
        yield str(specs['idleConsumption'])  # idleConsumption
        yield str(specs['maxConsumption'])  # maxConsumption
        
        # isOrchestrator - only first server (dc1) in centralized topology
        # We need to revert the index for checking if it was 0? 
        # The original code used 'server_index == 0' BEFORE increment? 
        # Wait, if we increment first, server_index will be 1 for the first server.
        # So we should use (server_index - 1) or check before incrementing.
        
        current_index = server_index - 1
        yield 'True' if current_index == 0 else 'False'
        
        # Location is handled separately by the XML generation function
        
        yield str(specs['cores'])  # cores
        yield str(specs['mips'])  # mips per core
        yield str(specs['ram'])  # ram
        yield str(specs['storage'])  # storage
        
        # Add server class as metadata for analysis
        yield server_class  # This will be added as a custom attribute
    
    return generator

# Create generator factories for both scenarios
def _20_server_spec_gen():
    return server_specification_generator(server_classes_20, server_specs_20)

def _100_server_spec_gen():
    return server_specification_generator(server_classes_100, server_specs_100)

print("Created heterogeneous server specification generators")


Created heterogeneous server specification generators


### AP Specification Generator

Access points -- network routing devices

In [25]:
def ap_specs_generator():
    yield 'True' # periphery
    yield '0' # idleConsumption
    yield '1' # maxConsumption
    yield 'False' # isOrchestrator
    yield '1' # cores
    yield '1' # mips
    yield '1' # ram
    yield '1' # storage
    yield 'AP' # class marker for APs

print("Created AP specs generator")

Created AP specs generator


### XML generation for server configurations

Functions create the XML structure with different server specifications 

In [26]:
def create_datacenter_names(AP_locs, server_locs):
    """
    Create unique names for servers and APs.

    Server names: dc1, dc2,...,dc{n}
    Access point: ap1, ap2,...,ap{n}
    """ 
    names = {}
    for i, loc in enumerate(AP_locs):
        names[loc] = f"ap{i+1}" 
    for i, loc in enumerate(server_locs):
        names[loc] = f"dc{i+1}"
    
    return names

def add_server_specifications(doc, server_locs, dc_names, server_specs_generator_factory, cluster_info):
    """
    Add servers to XML tree 

    Args: 
        doc: XML root element
        server_locs: list of (x,y) tuples for server locations 
        dc_names: dict mapping locations to names  
        server_specs_generator_factory: factory function for generating server specifications 
        cluster_info: dict with cluster and cluster head information (or None)
    """ 

    for i, s_loc in enumerate(server_locs): 
        gen = server_specs_generator_factory()

        dc_name = dc_names[s_loc] 
        s = etree.SubElement(doc, 'datacenter', name=dc_name)

        etree.SubElement(s, 'periphery').text = next(gen)
        etree.SubElement(s, 'idleConsumption').text = next(gen)
        etree.SubElement(s, 'maxConsumption').text = next(gen)
        etree.SubElement(s, 'isOrchestrator').text = next(gen)

        # Location 
        l = etree.SubElement(s, 'location')
        etree.SubElement(l, 'x_pos').text = str(s_loc[0])
        etree.SubElement(l, 'y_pos').text = str(s_loc[1])

        # Server specs
        etree.SubElement(s, 'cores').text = next(gen)
        etree.SubElement(s, 'mips').text = next(gen)
        etree.SubElement(s, 'ram').text = next(gen)
        etree.SubElement(s, 'storage').text = next(gen)

        # Server class metadata
        server_class = next(gen)
        etree.SubElement(s, 'serverClass').text = server_class

        # Cluster information
        if cluster_info is None: 
            # decentralized, each server is its own cluster
            etree.SubElement(s, 'cluster').text = str(i)
            etree.SubElement(s, 'clusterHead').text = 'True'
        else: 
            # hybrid or centralized: use provided cluster information
            dc = cluster_info[dc_name]
            cluster_id = dc['cluster']
            is_head = dc['head']
            etree.SubElement(s, 'cluster').text = str(cluster_id)
            etree.SubElement(s, 'clusterHead').text = str(is_head)
    
def add_AP_specifcation(doc, AP_locs, dc_names, ap_specs_generator):
    """ 
    Adds APs to XML tree
    """ 
    for ap_loc in AP_locs:
        gen = ap_specs_generator()
        ap_name = dc_names[ap_loc]
        ap = etree.SubElement(doc, 'datacenter', name=ap_name)
        
        etree.SubElement(ap, 'periphery').text = next(gen)
        etree.SubElement(ap, 'idleConsumption').text = next(gen)
        etree.SubElement(ap, 'maxConsumption').text = next(gen)
        etree.SubElement(ap, 'isOrchestrator').text = next(gen)
        
        l = etree.SubElement(ap, 'location')
        etree.SubElement(l, 'x_pos').text = str(ap_loc[0])
        etree.SubElement(l, 'y_pos').text = str(ap_loc[1])
        
        etree.SubElement(ap, 'cores').text = next(gen)
        etree.SubElement(ap, 'mips').text = next(gen)
        etree.SubElement(ap, 'ram').text = next(gen)
        etree.SubElement(ap, 'storage').text = next(gen)

def add_links(doc, connections, dc_names):
    """
    Adds network links to document tree
    """ 
    n = etree.SubElement(doc, 'network_links')
    for start_point, end_point in connections: 
        link = etree.SubElement(n, 'link')
        etree.SubElement(link, 'from').text = dc_names[start_point]
        etree.SubElement(link, 'to').text = dc_names[end_point]


def generate_file(AP_locs, server_locs, connections, server_specs_gen, ap_specs_gen, cluster_info=None):
    """ 
    Main function to generate a complete XML file
    """ 
    doc = etree.Element('edge_datacenters')

    # Filter out APs that are also in the server location
    only_APs = [ap for ap in AP_locs if ap not in server_locs]
    dc_names = create_datacenter_names(only_APs, server_locs)

    # Add components
    add_server_specifications(doc, server_locs, dc_names, server_specs_gen, cluster_info)
    add_AP_specifcation(doc, only_APs, dc_names, ap_specs_gen)
    add_links(doc, connections, dc_names)

    etree.indent(doc, space="    ")
    return etree.tostring(doc, encoding='UTF-8', xml_declaration=True, pretty_print=True)

print("XML generation functions defined")






    
    

def copy_baseline_files(dest_xml_path, topo, servers):
    """
    Copy baseline configuration files (cloud.xml, applications.xml, edge_devices.xml)
    from the homogeneous settings to the heterogeneous settings.
    """
    import os
    import shutil
    
    dest_dir = os.path.dirname(dest_xml_path)
    # Source based on standard naming: settings_{TOPOLOGY}_{SERVERS}servers
    # E.g. settings_H_100servers
    src_dir_name = f'settings_{topo}_{servers}servers'
    # Assuming the notebook is running in Environment_setup/revision/
    # and settings are in ../../EISim/EISim_settings/
    base_settings_path = '../../EISim/EISim_settings/'
    src_dir = os.path.join(base_settings_path, src_dir_name)
    
    files_to_copy = ['cloud.xml', 'applications.xml', 'edge_devices.xml']
    
    print(f"  Copying baseline files from {src_dir}...")
    for filename in files_to_copy:
        src = os.path.join(src_dir, filename)
        dst = os.path.join(dest_dir, filename)
        if os.path.exists(src):
            shutil.copy2(src, dst)
            # print(f"    Copied {filename}")
        else:
            print(f"    WARNING: Source file {filename} not found in {src_dir}")


XML generation functions defined


### Generate all configuration files


In [ ]:
print('Generate configuration files...')

print('Generating hybrid control topology files...')

# 100 servers hybrid topo config
hetero_100_hybrid_file = generate_file(AP_coords, 
                                    s_coords_100, links, 
                                    _100_server_spec_gen(), ap_specs_generator,
                                    server100_cluster_info_hybrid)


filename = '../../EISim/EISim_settings/revision/settings_H_100servers_hetero/edge_datacenters.xml'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'wb') as f: 
    f.write(hetero_100_hybrid_file)
print(f"Created: {filename}")
copy_baseline_files(filename, 'H', 100)

# 20 servers hybrid topo config

hetero_20_hybrid_file = generate_file(AP_coords, 
                                    s_coords_20, links, 
                                    _20_server_spec_gen(), ap_specs_generator,
                                    server20_cluster_info_hybrid)


filename = '../../EISim/EISim_settings/revision/settings_H_20servers_hetero/edge_datacenters.xml'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'wb') as f: 
    f.write(hetero_20_hybrid_file)
print(f"Created: {filename}")
copy_baseline_files(filename, 'H', 20)

print("Generating centralized control topology files...")
hetero_100_central_file = generate_file(AP_coords, 
                                        s_coords_100, 
                                        links, 
                                        _100_server_spec_gen(), 
                                        ap_specs_generator, server100_cluster_info_centralized)
filename = '../../EISim/EISim_settings/revision/settings_C_100servers_hetero/edge_datacenters.xml'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'wb') as f: 
    f.write(hetero_100_central_file)
print(f"Created: {filename}")
copy_baseline_files(filename, 'C', 100)


hetero_20_central_file = generate_file(AP_coords, 
                                        s_coords_20, 
                                        links, 
                                        _20_server_spec_gen(), 
                                        ap_specs_generator, server20_cluster_info_centralized)
filename = '../../EISim/EISim_settings/revision/settings_C_20servers_hetero/edge_datacenters.xml'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'wb') as f: 
    f.write(hetero_20_central_file)
print(f"Created: {filename}")
copy_baseline_files(filename, 'C', 20)


print("Generating decentralized control topology files...")
# For decentralized: pass cluster_info=None so each server is its own cluster
hetero_100_dec_file = generate_file(AP_coords, 
                                    s_coords_100, 
                                    links, 
                                    _100_server_spec_gen(), 
                                    ap_specs_generator, 
                                    None)  # None = decentralized (each server is its own cluster)
filename = '../../EISim/EISim_settings/revision/settings_D_100servers_hetero/edge_datacenters.xml'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'wb') as f: 
    f.write(hetero_100_dec_file)
print(f"Created: {filename}")
copy_baseline_files(filename, 'D', 100)


hetero_20_dec_file = generate_file(AP_coords, 
                                    s_coords_20, 
                                    links, 
                                    _20_server_spec_gen(), 
                                    ap_specs_generator, 
                                    None)  # None = decentralized (each server is its own cluster)
filename = '../../EISim/EISim_settings/revision/settings_D_20servers_hetero/edge_datacenters.xml'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'wb') as f: 
    f.write(hetero_20_dec_file)
print(f"Created: {filename}")
copy_baseline_files(filename, 'D', 20)


Generate configuration files...
Generating hybrid control topology files...
Created: ../../EISim/EISim_settings/tmp/settings_H_100servers_hetero/edge_datacenters.xml
  Copying baseline files from ../../EISim/EISim_settings/settings_H_100servers...
Created: ../../EISim/EISim_settings/tmp/settings_H_20servers_hetero/edge_datacenters.xml
  Copying baseline files from ../../EISim/EISim_settings/settings_H_20servers...
Generating centralized control topology files...
Created: ../../EISim/EISim_settings/tmp/settings_C_100servers_hetero/edge_datacenters.xml
  Copying baseline files from ../../EISim/EISim_settings/settings_C_100servers...
Created: ../../EISim/EISim_settings/tmp/settings_C_20servers_hetero/edge_datacenters.xml
  Copying baseline files from ../../EISim/EISim_settings/settings_C_20servers...
Generating decentralized control topology files...
Created: ../../EISim/EISim_settings/tmp/settings_D_100servers_hetero/edge_datacenters.xml
  Copying baseline files from ../../EISim/EISim_se